In [ ]:
# load libraries
library(glmGamPoi)
library(Matrix)
library(nebula)
library(SingleCellExperiment)
library(Seurat)
library(tidyverse)
library(data.table)
library(matrixStats)

In [ ]:
# convenience wrapper around nebula and glmGamPoi
nebula.mult <- function(formula, count, df){
    pred <- model.matrix(formula, data=df)
    sid <- df$id
    fit.nebula <- nebula::nebula(
        count,
        sid,
        offset=colMeans2(count),
        pred=pred,
        cpc=0,
        mincp=0
        )
    
    fit.result <- fit.nebula$summary
    rownames(fit.result) <- fit.result$gene
    result.nona <- fit.result %>%
                    mutate(
                        Estimate=logFC_tx_cell,
                        'Std. Error'=se_tx_cell,
                        't value'=logFC_tx_cell/se_tx_cell,
                        'Pr(>|t|)'=p_tx_cell
                    ) %>%
                    select(Estimate, 'Std. Error', 't value', 'Pr(>|t|)')
    result.na <- data.frame(matrix(nrow=nrow(count), ncol=4), row.names=rownames(count))
    colnames(result.na) <- colnames(result.nona)
    result.na[rownames(result.nona),] <- result.nona
    return(result.na)
    }

glmgp.default <- function(formula, count, df){
    sce.obj <- SingleCellExperiment::SingleCellExperiment(list(counts=count), colData=df)
    sce.pb <- glmGamPoi::pseudobulk(
        sce.obj,
        group_by=vars(id, tx_cell),
        verbose=FALSE
        )

    fit <- glmGamPoi::glm_gp(sce.pb, design=~1+tx_cell)
    test <- glmGamPoi::test_de(fit, reduced_design=~1)
    
    beta <- fit$Beta[,'tx_cell']
    pval <- test$pval
    tval <- qnorm(1-pval/2) * sign(beta)
    se <- beta/tval
    result <- cbind(beta, se, tval, pval)
    colnames(result) <- c('Estimate', 'Std. Error', 't value', 'Pr(>|t|)')
    return(result)
    }

In [ ]:
# load data
obj <- base::readRDS('datasets/yazar.seurat.rds')

In [ ]:
# some parameters
n.celltype <- 5 # top 5 cell types
n.cell <- 100 # minimum number of cells per donor per cell type

# select cell-types and donors
cols <- c('donor_id', 'cell_type')
ct.used <- obj[[cols]] %>% 
    group_by(cell_type) %>%
    summarise(n=n()) %>%
    arrange(desc(n)) %>%
    top_n(n.celltype) %>%
    pull(cell_type)

donor.used <- obj[[cols]] %>%
    group_by(donor_id, cell_type, .drop=FALSE) %>%
    summarise(n=n()) %>%
    pivot_wider(names_from=cell_type, values_from=n) %>%
    select(all_of(ct.used)) %>%
    filter(if_all(ct.used,~.>n.cell)) %>% # ~>.10 is purrr style lambda function
    pull(donor_id) 

In [ ]:
obj.used <- obj %>% subset(cell_type %in% ct.used) %>% subset(donor_id %in% donor.used)
idx.order <- obj.used[['donor_id']] %>% arrange(donor_id) %>% rownames_to_column() %>% pull(rowname)
cnt.used <- GetAssayData(obj.used, slot = "data")[,idx.order]
col.data.used <- obj.used[[]][idx.order,]

In [ ]:
for (iter.ct in 1:5){
    # stratify by cell type
    cell.type <- ct.used[[iter.ct]]
    idx.cell.type <- which(col.data.used$cell_type == cell.type)
    
    cnt.cell.type <- cnt.used[,idx.cell.type]
    cnt.cell.type <- cnt.cell.type[rowMeans2(cnt.cell.type) > 0.1,]
    col.data.cell.type <- col.data.used[idx.cell.type,]
    
    cnt.list <- list()
    for (donor in donor.used){
        cell.donor <- which(col.data.cell.type$donor_id == donor)
        cnt.list[[donor]] <- cnt.cell.type[,cell.donor]
        }
    
    col.data.list <- list()
    for (donor in donor.used){
        cell.donor <- rownames(col.data.cell.type)[which(col.data.cell.type$donor_id == donor)]
        col.data.list[[donor]] <- col.data.cell.type[cell.donor,]
        }

    n.cell.per.donor <- 10
    for (n.donor.sample in c(10, 20, 40)){
    
        result.list.pb <- list()
        result.list.glmm <- list()
        
        for (iter in 1:100){
            # select donor
            donor.sample <- as.character(sample(donor.used, n.donor.sample, replace=FALSE))
            donor.group <- sample(rep(c(0,1), n.donor.sample/2), n.donor.sample) # assign group status
            
            # select n.cell.per.donor cells from each donor
            cell.sample <- lapply(col.data.list[donor.sample], function(df) { rownames(df) }) 
            cell.subset.sample <- lapply(cell.sample, function(cell.per.donor) { sample(cell.per.donor, rbinom(n=1, size=n.cell.per.donor, p=0.5) + n.cell.per.donor/2 , replace=FALSE) })
            
            # subset count
            cnt.sample <- do.call(cbind, cnt.list[donor.sample])[,do.call(c, cell.subset.sample)]
            
            # subset col.data
            col.data.sample <- col.data.list[donor.sample]
            names(col.data.sample) <- NULL
            col.data.sample <- do.call(rbind, col.data.sample)[do.call(c, cell.subset.sample),]
            
            # assign tx status
            names(donor.group) <- donor.sample
            col.data.sample$tx_cell <- donor.group[as.character(col.data.sample$donor_id)]
            col.data.sample$id <- col.data.sample$donor_id
        
            # fit models
            result.list.pb[[iter]] <- glmgp.default(as.formula('~tx_cell'), cnt.sample, col.data.sample)
            result.list.glmm[[iter]] <- nebula.mult(as.formula('~tx_cell'), cnt.sample, col.data.sample)
        
            }
        saveRDS(result.list.pb, paste('results/pb','n',n.donor.sample,'ct',iter.ct,'rds',sep='.'))
        saveRDS(result.list.glmm, paste('results/glmm','n',n.donor.sample,'ct',iter.ct,'rds',sep='.') )
        }
    }
         